In [1]:
import numpy as np
import matplotlib.pyplot as plt
    
from openai import OpenAI
from dotenv import load_dotenv
from typing import List, Dict

from datasets import load_dataset

import utils as utils
from transformers import AutoProcessor, SeamlessM4TModel

/home/romit/.virtualenvs/mahout/lib/python3.11/site-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(


Huggingface models

In [2]:
model_id = 'facebook/hf-seamless-m4t-large'

In [3]:
processor = AutoProcessor.from_pretrained(model_id)
model = SeamlessM4TModel.from_pretrained(model_id, device_map='auto')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/home/romit/.virtualenvs/mahout/lib/python3.11/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


generation_config.json:   0%|          | 0.00/3.35k [00:00<?, ?B/s]

Hugginface open datasets

In [4]:
dataset = load_dataset('HuggingFaceH4/no_robots')

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train_sft split:   0%|          | 0/9500 [00:00<?, ? examples/s]

Generating test_sft split:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
def get_stringify_conversation(example: Dict):
    str_msg = ""
    for message in example['messages']:
        str_msg += message['content']
        str_msg += '\n'

    example['str_msg'] = str_msg
    return example

In [ ]:
dataset = dataset.map(
    utils.get_stringify_conversation,
    num_proc=20
)

In [ ]:
print(dataset['train_sft']['str_msg'][0])

In [ ]:
m = dataset['train_sft']['str_msg'][0]

In [ ]:
# m = 'Translate english to devnagri hindi: ' + m

In [ ]:
m = "My name is Romit"

In [ ]:
tokenizer.src_lang = 'en_XX'
t = tokenizer(m, return_tensors='pt').input_ids.to('cuda:0')

In [ ]:
o = model.generate(t, forced_bos_token_id=tokenizer.lang_code_to_id['hi_IN'])

In [ ]:
tokenizer.decode(o[0])

In [ ]:
article_hi = "संयुक्त राष्ट्र के प्रमुख का कहना है कि सीरिया में कोई सैन्य समाधान नहीं है"
article_ar = "الأمين العام للأمم المتحدة يقول إنه لا يوجد حل عسكري في سوريا."

In [ ]:
tokenizer.src_lang = "en_XX"
encoded_hi = tokenizer(m, return_tensors="pt")
generated_tokens = model.generate(**encoded_hi, forced_bos_token_id=tokenizer.lang_code_to_id["hi_IN"])
tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)